# PyTorch Lightning abstraction basics

*Putting it all together with PL abstraction mechanics.*

[PyTorch Lightning](https://www.pytorchlightning.ai/) is a framework built on top of PyTorch, which takes care of the boilerplates, as well as simplifies the training parallelism. It is often compared to Keras for TensorFlow. We will integrated directly our models and data modules with the PL mechanics, so that distributed training becomes easier.

Lightning has grown into a massive framework with functionalities missing from vanilla PyTorch, but for basic-level understanding of the abstraction logic, there are only a few usefull components, most notably:
* `pl.LightningModule`, a wrapper for a PyTorch model, with implementable train, test, and validation loops
* `pl.LightningDataModule`, a wrapper for a PyTorch `Dataset`, with implementable data splitting logic
* `pl.Trainer` to orchestrate training + testing phases, as well as inference. Gradient clipping, 
* `pl.callbacks.base.Callback` to organize runtime workflow. Comes standard with `EarlyStopping`, `ModelCheckpoint`, `LearningRateMonitor`, and `ModelPruning`, among others
* A Profiler to debug resource utilization

Let's first load all the necessary params.

In [1]:
from kosmoss import CONFIG, LOGS_PATH, METADATA
from kosmoss.parallel.data import FlattenedDataModule
from kosmoss.parallel.mlp import LitMLP

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

# Ensures this Notebook's reproducibility
pl.seed_everything(42, workers=True)

step = CONFIG['timestep']
params = METADATA[str(step)]['flattened']

Global seed set to 42


## Model and training logic

In [3]:
!cat mlp.py

import os.path as osp
from pytorch_lightning import LightningModule
import torch
import torch.nn as nn
import torch_optimizer as optim
import torchmetrics.functional as F
from typing import List, Tuple, Union

from kosmoss import CONFIG, DATA_PATH

class ThreeDCorrectionModule(LightningModule):
    
    def __init__(self):
        super().__init__()
        
        # 'The LightningModule knows what device it is on. You can access the reference via self.device. Sometimes it is necessary to store tensors as module attributes. However, if they are not parameters they will remain on the CPU even if the module gets moved to a new device. To prevent that and remain device agnostic, register the tensor as a buffer in your modules’s __init__ method with register_buffer().'
        self.register_buffer("epsilon", torch.tensor(1.e-8))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)
    
    def _common_step(self, 
                     batch: List[torch.Tensor

In [4]:
x_feats = params['x_shape'][-1]
y_feats = params['y_shape'][-1]

In [5]:
print(f'x number of features: {x_feats}')
print(f'y number of features: {y_feats}')

x number of features: 4128
y number of features: 552


In [6]:
mlp = LitMLP(
    in_channels=x_feats,
    hidden_channels=100,
    out_channels=y_feats
)
mlp

LitMLP(
  (normalization_layer): Normalize()
  (feature_selection_layer): SelectFeatures()
  (net): Sequential(
    (0): Normalize()
    (1): SelectFeatures()
    (2): Linear(in_features=4128, out_features=100, bias=True)
    (3): SiLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): SiLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): SiLU()
    (8): Linear(in_features=100, out_features=552, bias=True)
  )
)

## Dataset creation and data loading mechanics

In [7]:
!cat data.py

import numpy as np
import os.path as osp
from pytorch_lightning import LightningDataModule
import torch
from typing import Tuple, Union

from kosmoss import CONFIG, METADATA, PROCESSED_DATA_PATH

class FlattenedDataset(torch.utils.data.Dataset):
    
    def __init__(self, 
                 step: int, 
                 mode: Union['efficient', 'controlled'] = 'controlled') -> None:
        super().__init__()
        self.step = step
        self.mode = mode
        self.params = METADATA[str(self.step)]['flattened']
    
    def __len__(self) -> int:
        
        return self.params['dataset_len']
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor]:
        
        shard_size = len(self) // self.params['num_shards']
        fileidx = idx // shard_size
        rowidx = idx % shard_size
        
        def _load(name: Union['x', 'y']) -> Tuple[torch.Tensor]:
            main_path = osp.join(PROCESSED_DATA_PATH, f"flattened-{self.step}")
            
            if self.m

* `batch_size` sets the number of element in a batch of data.
* `num_workers` sets the number of workers the DataLoader can spawn to handle data loading and Dataset batching.

In [8]:
import psutil
cores = psutil.cpu_count(logical=False)

In [9]:
datamodule = FlattenedDataModule(
    batch_size=1024,
    
    # In CPU-only setup, make sure you still have enough cores to handle the training, 
    # Not just data loading, otherwise, it will bottleneck
    num_workers=cores
)

## Orchestrating the training

In [10]:
logger = TensorBoardLogger(
    save_dir=LOGS_PATH,
    name='flattened_mlp_logs',
    log_graph=True
)

All the training instrumentation is done by an object call the Trainer. You can fix parameters such as:
* `max_epochs` unless an early stopping happens
* `accelerator` type and `device` logical number

Notably interesting: 
* `callbacks` to handle in-betweens
* `gradient_clip_val` and `gradient_clip_algorithm` to setup the gradient clipping
* `logger` to interface with loss and metrics logging
* `resume_from_checkpoint` helps resuming a previously initiated training
* `amp_backend` to switch to Nvidia Apex framework for Automatic Mixed Precision support

In [11]:
cpu_trainer = Trainer(
    max_epochs=1,
    logger=logger,
    deterministic=True,
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


Training CPU is a one-line

In [12]:
cpu_trainer.fit(model=mlp, datamodule=datamodule)


  | Name                    | Type           | Params
-----------------------------------------------------------
0 | normalization_layer     | Normalize      | 0     
1 | feature_selection_layer | SelectFeatures | 0     
2 | net                     | Sequential     | 488 K 
-----------------------------------------------------------
488 K     Trainable params
0         Non-trainable params
488 K     Total params
1.955     Total estimated model params size (MB)


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/tensorboard.py:247: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  rank_zero_warn(


Validation sanity check:   0% 0/2 [00:00<?, ?it/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: Caught RuntimeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/auto_restart.py", line 474, in _capture_metadata_collate
    data = default_collate(samples)
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/_utils/collate.py", line 54, in defau

Never forget to test. The handy thing with the `Trainer` is, if a `.test()` is called somewhere at runtime, once a `SIGTERM` is thrown by the runtime such as a `KeyboardInterruptError`, it gets caught by Lightning, which tries to gracefully release resources, terminate training, and run the test anyway.

In [ ]:
cpu_trainer.test(model=mlp, datamodule=datamodule)

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 1823, in _run_once
    event_list = self._selector.select(timeout)
  File "/usr/lib/python3.8/selectors.py",